In [68]:
from astropy.io import fits
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import subprocess
import os

In [31]:
# Open fits file and assign to variable.
datafile = fits.open('DR7_tot_6_0.fits')

In [32]:
# Inspect fits file summary to see which dataset to use by passing .info() method on the file variable.
datafile.info()

Filename: DR7_tot_6_0.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1                1 BinTableHDU    271   70841R x 127C   [I, J, I, 22A, D, D, 5E, 5E, 5E, D, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, I, E, E, I, I, I, I, I, E, E, E, E, I, E, I, E, E, D, D, D, D, E, E, E, E, E, E, D, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, I, I, I, I, I, I, I, E, E, E, E, E, E, E, E, E, E, E, E, I, E, E, E, E, E, E, I, E, E]   


In [33]:
# Assign header (dataset) in fits file to a variable. Inspect the column names by passing the .header method 
datahdr = datafile[1]
datahdr.header

XTENSION= 'BINTABLE'           /Binary table written by MWRFITS v0.95           
BITPIX  =                    8 /Required value                                  
NAXIS   =                    2 /Required value                                  
NAXIS1  =                  568 /Number of bytes per row                         
NAXIS2  =                70841 /Number of rows                                  
PCOUNT  =                    0 /Normally 0 (no varying arrays)                  
GCOUNT  =                    1 /Required value                                  
TFIELDS =                  127 /Number of columns in table                      
COMMENT                                                                         
COMMENT  *** Column formats ***                                                 
COMMENT                                                                         
TFORM1  = 'I       '           /                                                
TFORM2  = 'J       '        

In [34]:
# Assign data columns to variables 
westplate   = datahdr.data['plate']  
westmjd     = datahdr.data['mjd']
westfiber   = datahdr.data['fiber']
westEWHA    = datahdr.data['EWHA']
westEWHAerr = datahdr.data['EWHAERR']

In [35]:
# Entries in these columns are 16bit integer types but want to convert them to strings in order to later assemble
# them per the sloan object id description (i.e 'spSpec-<mjd>-<plate#>-<fiber#>')
strplate = [] 
strmjd   = []
strfiber = []
for i in westplate:
    strplate.append(str(i))

for i in westmjd:
    strmjd.append(str(i))
    
for i in westfiber:
    strfiber.append(str(i))

In [24]:
# The object id fiber number is always expressed as a zero padded 4-digit number. Here I write a loop to pad
# the entries with zeros based on the entries length.
#zfilledfiber = []
#for i in strfiber:
#    if len(i) == 3:
#        i = '0' + i
#    elif len(i) == 2:
#        i = '00' + i
#    elif len(i) == 1:
#        i = '000' + i
#    zfilledfiber.append(i)
    
#zfilledplate = []
#for i in strplate:
#    if len(i) == 3:
#        i = i + '0' 
#    elif len(i) == 2:
#        i = i + '00'
#    elif len(i) == 1:
#        i = i + '000'
#    zfilledplate.append(i)
    
    
#print(zfilledfiber)

In [43]:
# multi-index for-loop to combine the column entries and reexpress in the form of the sloan DR7 object-id.
objectid  = []
querylist = []
for i,j,k in zip(strplate,strmjd,strfiber):
    objectid.append('spSpec-' + j + '-' + i + '-' + k)
    querylist.append(i + ' ' + j + ' ' + k)

    print(objectid)

['2212 53789 356', '2212 53789 489', '2335 53730 185', '2340 53733 84', '2335 53730 383', '2783 54524 317', '2783 54524 128', '2783 54524 98', '2784 54529 92', '2784 54529 6', '1096 52974 510', '1096 52974 495', '1096 52974 483', '1096 52974 494', '1096 52974 588', '1096 52974 496', '1095 52521 347', '1095 52521 389', '1095 52521 422', '1095 52521 463', '1096 52974 621', '1095 52521 477', '1096 52974 629', '1095 52521 600', '1095 52521 584', '1095 52521 582', '1094 52524 353', '1094 52524 469', '1094 52524 382', '1094 52524 541', '1094 52524 491', '1094 52524 500', '1094 52524 483', '1094 52524 504', '1094 52524 546', '1094 52524 544', '1094 52524 548', '1094 52524 503', '1094 52524 510', '1094 52524 501', '1093 52591 349', '1094 52524 482', '1093 52591 360', '1094 52524 631', '1094 52524 516', '1093 52591 383', '1093 52591 389', '1093 52591 463', '1094 52524 630', '1093 52591 476', '1093 52591 391', '2214 53794 483', '2216 53795 422', '2216 53795 424', '2218 53816 341', '2216 53795 48

In [45]:
# Now that the singular plate,mjd,and fiber columns have been correctly formatted into object id's 
# I'll then construct a simple dataframe for the Halpha equivalent widths of the west et al 2011 Mdwarf sample
# for use in comparing my equivalent width code results against.
datframe = pd.DataFrame(index=None)
datframe['Sloan Object ID']  = objectid
datframe['West2011 EWHA']    = westEWHA
datframe['West2011 EWHAerr'] = westEWHAerr


#print(datframe)

In [50]:
allobjtest1000 = datframe.sample(n=1000)

In [49]:
#dMs100test

In [52]:
West2011_DLlist_1000obj = allobjtest1000['Sloan Object ID'].to_csv('West2011sample1000_querylist.txt',index=False)

In [57]:
path = os.getcwd() + '/testEWspectra/'

In [58]:
path

'/Users/jventura/code_abode/python/EqWi/halphatest/testEWspectra/'

In [73]:
for i in os.listdir(path):
    if i[-4:] == '.fit': 
        with fits.open(os.path.join(path+i)) as hdulist:
            prihdr = hdulist[0]
            obtype = prihdr.header['OBJTYPE']
            if obtype == 'STAR_RED_DWARF':
                subprocess.run(['mv', path+i , path+'/mdwarfs/']) 

MC1TEMDN=-0.00000000000000E+00 / sp1 mech median temp                            [astropy.io.fits.card]
MC1TBCB =-0.00000000000000E+00 / sp1 mech Temp_Blue_Cam_Bot                      [astropy.io.fits.card]


In [1]:
from linetools import eqwidth

IndentationError: unindent does not match any outer indentation level (linetools.py, line 88)